In [105]:
import pandas as pd
import os

def read_and_concatenate(folder_path):
    # Lists to store DataFrames
    gen_dataframes = []
    load_dataframes = []

    # Iterate over all files in the folder
    for file in os.listdir(folder_path):
        if file.endswith('.csv'):
            file_path = os.path.join(folder_path, file)

            # Read 'gen' files
            if file.startswith('gen'):
                df = pd.read_csv(file_path)
                df.fillna(0, inplace=True)
                gen_dataframes.append(df)

            # Read 'load' files
            elif file.startswith('load'):
                df = pd.read_csv(file_path)
                df.fillna(0, inplace=True)
                load_dataframes.append(df)

    # Concatenate DataFrames vertically
    gen_concatenated = pd.concat(gen_dataframes, axis=0, ignore_index=True)
    load_concatenated = pd.concat(load_dataframes, axis=0, ignore_index=True)

    # Combine 'gen' and 'load' DataFrames
    combined_dataframe = pd.concat([gen_concatenated, load_concatenated], axis=0, ignore_index=True)

    return combined_dataframe

# Provide the path to your folder containing the CSV files
folder_path = '../data/raw_data/'
data = read_and_concatenate(folder_path)

# Optionally, save the combined DataFrame to a new CSV file
#combined_df.to_csv('combined_dataset.csv', index=False)


In [106]:
data['StartTime'] = pd.to_datetime(data['StartTime'].str.replace('\+00:00Z', '', regex=True)).dt.strftime('%Y-%m-%d %H:%M:%S')
data['EndTime'] = pd.to_datetime(data['EndTime'].str.replace('\+00:00Z', '', regex=True)).dt.strftime('%Y-%m-%d %H:%M:%S')
data['StartTime'] = pd.to_datetime(data['StartTime'])
data['EndTime'] = pd.to_datetime(data['EndTime'])

In [107]:
data['AreaID'] = data['AreaID'].replace({
    '10YHU-MAVIR----U': 'HU',
    '10YIT-GRTN-----B': 'IT',
    '10YPL-AREA-----S': 'PO',
    '10YES-REE------0': 'SP',
    '10Y1001A1001A92E': 'UK',
    '10Y1001A1001A83F': 'DE',
    '10Y1001A1001A65H': 'DK',
    '10YSE-1--------K': 'SE',
    '10YNL----------L': 'NE'
})

In [108]:
data

,StartTime,EndTime,AreaID,UnitName,PsrType,quantity,Load
0,2021-12-31 23:45:00,2022-01-01 00:00:00,DE,MAW,B01,4333.0,NaN
1,2022-01-01 00:00:00,2022-01-01 00:15:00,DE,MAW,B01,4325.0,NaN
2,2022-01-01 00:15:00,2022-01-01 00:30:00,DE,MAW,B01,4319.0,NaN
3,2022-01-01 00:30:00,2022-01-01 00:45:00,DE,MAW,B01,4323.0,NaN
4,2022-01-01 00:45:00,2022-01-01 01:00:00,DE,MAW,B01,4328.0,NaN
...,...,...,...,...,...,...,...
1645660,2022-07-18 05:00:00,2022-07-18 05:30:00,UK,MAW,NaN,NaN,631.0
1645661,2022-07-18 05:30:00,2022-07-18 06:00:00,UK,MAW,NaN,NaN,696.0
1645662,2022-07-18 06:00:00,2022-07-18 06:30:00,UK,MAW,NaN,NaN,770.0
1645663,2022-07-18 06:30:00,2022-07-18 07:00:00,UK,MAW,NaN,NaN,840.0


In [109]:
data.fillna(0, inplace=True)
data['gen/load']='load'
data['Load'] = data['Load'].fillna(0)
data.loc[data['Load']==0,'gen/load']='gen'

data['power']=data['quantity']+data['Load']
data

,StartTime,EndTime,AreaID,UnitName,PsrType,quantity,Load,gen/load,power
0,2021-12-31 23:45:00,2022-01-01 00:00:00,DE,MAW,B01,4333.0,0.0,gen,4333.0
1,2022-01-01 00:00:00,2022-01-01 00:15:00,DE,MAW,B01,4325.0,0.0,gen,4325.0
2,2022-01-01 00:15:00,2022-01-01 00:30:00,DE,MAW,B01,4319.0,0.0,gen,4319.0
3,2022-01-01 00:30:00,2022-01-01 00:45:00,DE,MAW,B01,4323.0,0.0,gen,4323.0
4,2022-01-01 00:45:00,2022-01-01 01:00:00,DE,MAW,B01,4328.0,0.0,gen,4328.0
...,...,...,...,...,...,...,...,...,...
1645660,2022-07-18 05:00:00,2022-07-18 05:30:00,UK,MAW,0,0.0,631.0,load,631.0
1645661,2022-07-18 05:30:00,2022-07-18 06:00:00,UK,MAW,0,0.0,696.0,load,696.0
1645662,2022-07-18 06:00:00,2022-07-18 06:30:00,UK,MAW,0,0.0,770.0,load,770.0
1645663,2022-07-18 06:30:00,2022-07-18 07:00:00,UK,MAW,0,0.0,840.0,load,840.0


In [112]:
# Extract date and hour
data['Date'] = data['StartTime'].dt.date
data['Hour'] = data['StartTime'].dt.hour
data

,StartTime,EndTime,AreaID,UnitName,PsrType,quantity,Load,gen/load,power,Date,Hour
0,2021-12-31 23:45:00,2022-01-01 00:00:00,DE,MAW,B01,4333.0,0.0,gen,4333.0,2021-12-31,23
1,2022-01-01 00:00:00,2022-01-01 00:15:00,DE,MAW,B01,4325.0,0.0,gen,4325.0,2022-01-01,0
2,2022-01-01 00:15:00,2022-01-01 00:30:00,DE,MAW,B01,4319.0,0.0,gen,4319.0,2022-01-01,0
3,2022-01-01 00:30:00,2022-01-01 00:45:00,DE,MAW,B01,4323.0,0.0,gen,4323.0,2022-01-01,0
4,2022-01-01 00:45:00,2022-01-01 01:00:00,DE,MAW,B01,4328.0,0.0,gen,4328.0,2022-01-01,0
...,...,...,...,...,...,...,...,...,...,...,...
1645660,2022-07-18 05:00:00,2022-07-18 05:30:00,UK,MAW,0,0.0,631.0,load,631.0,2022-07-18,5
1645661,2022-07-18 05:30:00,2022-07-18 06:00:00,UK,MAW,0,0.0,696.0,load,696.0,2022-07-18,5
1645662,2022-07-18 06:00:00,2022-07-18 06:30:00,UK,MAW,0,0.0,770.0,load,770.0,2022-07-18,6
1645663,2022-07-18 06:30:00,2022-07-18 07:00:00,UK,MAW,0,0.0,840.0,load,840.0,2022-07-18,6


In [115]:
aggregated_data['AreaID']

0          0
1          0
2          0
3          0
4          0
          ..
157331    UK
157332    UK
157333    UK
157334    UK
157335    UK
Name: AreaID, Length: 157336, dtype: object

In [114]:
aggregated_data = data.groupby(['AreaID', 'gen/load', 'Date', 'Hour'])['power'].sum().reset_index()

aggregated_data['concatenated'] = aggregated_data['AreaID']  + aggregated_data['gen/load']

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [ ]:
pivot = aggregated_data.pivot_table(
index=['Date', 'Hour'],
columns=['concatenated'],
values='power',
aggfunc='sum'
)
pivot

KeyError: 'concatenated'

In [102]:
pivot.dropna()

concatenated           0      DEDE    DKDK     HUHU     ITIT     NENE  \
Date       Hour                                                         
2021-12-31 23     4180.0   79250.0  6966.0   4539.0  26819.0  13861.0   
2022-01-01 0     14291.0  310282.0  6823.0  17833.0  25370.0  53414.0   
           1     16595.0  298816.0  6435.0  16952.0  24213.0  50562.0   
           2     19011.0  290652.0  6123.0  16341.0  23796.0  50205.0   
           3     18141.0  284027.0  5896.0  15917.0  23826.0  49456.0   
...                  ...       ...     ...      ...      ...      ...   
2023-01-01 18     7526.0  322449.0  8098.0  18910.0  30808.0  57507.0   
           19     8398.0  308616.0  7675.0  18540.0  28489.0  56348.0   
           20     7269.0  302228.0  7329.0  17789.0  26611.0  53315.0   
           21     6247.0  287825.0  6727.0  16917.0  24227.0  49809.0   
           22    11355.0  268231.0  5934.0  15896.0  22462.0  49000.0   

concatenated        POPO     SESE      SPSP    UKUK  
Date       Hour                                      
2021-12-31 23    19079.0  26391.0   30360.0   677.0  
2022-01-01 0     18426.0  26438.0   28473.0  1244.0  
           1     18015.0  26306.0   27008.0  1131.0  
           2     17965.0  25659.0   26128.0  1091.0  
           3     17923.0  26157.0   25686.0   969.0  
...                  ...      ...       ...     ...  
2023-01-01 18    21674.0  29302.0  161892.0    39.0  
           19    20436.0  28492.0  168416.0    55.0  
           20    18918.0  27855.0  168652.0    45.0  
           21    17274.0  26370.0  159032.0    37.0  
           22    16297.0  25212.0  144688.0    41.0  

[6427 rows x 10 columns]

In [96]:
pivot.columns

MultiIndex([(   0,  'gen'),
            ('DE',  'gen'),
            ('DE', 'load'),
            ('DK',  'gen'),
            ('DK', 'load'),
            ('HU',  'gen'),
            ('HU', 'load'),
            ('IT',  'gen'),
            ('IT', 'load'),
            ('NE',  'gen'),
            ('NE', 'load'),
            ('PO',  'gen'),
            ('PO', 'load'),
            ('SE',  'gen'),
            ('SE', 'load'),
            ('SP',  'gen'),
            ('SP', 'load'),
            ('UK',  'gen'),
            ('UK', 'load')],
           names=['AreaID', 'gen/load'])

In [97]:
pivot.columns = ['_'.join(col).strip() for col in pivot.columns.values]

TypeError: sequence item 0: expected str instance, int found

In [ ]:
pivot

AreaID                 0 10Y1001A1001A65H         10Y1001A1001A83F            \
gen/load               1                0       1                0         1   
Date       Hour                                                                
2021-12-31 23     4180.0           3314.0  3652.0          42196.0   37054.0   
2022-01-01 0     14291.0           3218.0  3605.0         165125.0  145157.0   
           1     16595.0           3126.0  3309.0         160415.0  138401.0   
           2     19011.0           3080.0  3043.0         158035.0  132617.0   
           3     18141.0           3044.0  2852.0         157016.0  127011.0   
...                  ...              ...     ...              ...       ...   
2023-01-01 19     8398.0           3507.0  4168.0         177225.0  131391.0   
           20     7269.0           3354.0  3975.0         172484.0  129744.0   
           21     6247.0           3229.0  3498.0         166627.0  121198.0   
           22    11355.0           3055.0  2879.0         153785.0  114446.0   
           23    16985.0              NaN     NaN         110140.0   81443.0   

AreaID          10Y1001A1001A92E       10YES-REE------0           \
gen/load                       0     1                0        1   
Date       Hour                                                    
2021-12-31 23              677.0   NaN          20827.0   9533.0   
2022-01-01 0              1244.0   NaN          19530.0   8943.0   
           1              1131.0   NaN          18383.0   8625.0   
           2              1091.0   NaN          17680.0   8448.0   
           3               969.0   NaN          17396.0   8290.0   
...                          ...   ...              ...      ...   
2023-01-01 19                NaN  55.0          98224.0  70192.0   
           20                NaN  45.0          97620.0  71032.0   
           21                NaN  37.0          91896.0  67136.0   
           22                NaN  41.0          84324.0  60364.0   
           23                NaN  32.0          59140.0  44148.0   

AreaID          10YHU-MAVIR----U         10YIT-GRTN-----B          \
gen/load                       0       1                0       1   
Date       Hour                                                     
2021-12-31 23             4254.0   285.0          21121.0  5698.0   
2022-01-01 0             16457.0  1376.0          19756.0  5614.0   
           1             15426.0  1526.0          18685.0  5528.0   
           2             14781.0  1560.0          18124.0  5672.0   
           3             14630.0  1287.0          18400.0  5426.0   
...                          ...     ...              ...     ...   
2023-01-01 19            17873.0   667.0          24617.0  3872.0   
           20            17141.0   648.0          22944.0  3667.0   
           21            16259.0   658.0          20791.0  3436.0   
           22            15273.0   623.0          19198.0  3264.0   
           23            10900.0   468.0              NaN     NaN   

AreaID          10YNL----------L          10YPL-AREA-----S          \
gen/load                       0        1                0       1   
Date       Hour                                                      
2021-12-31 23            10326.0   3535.0          14438.0  4641.0   
2022-01-01 0             40706.0  12708.0          13935.0  4491.0   
           1             39465.0  11097.0          13579.0  4436.0   
           2             38923.0  11282.0          13397.0  4568.0   
           3             38211.0  11245.0          13364.0  4559.0   
...                          ...      ...              ...     ...   
2023-01-01 19            47635.0   8713.0          15468.0  4968.0   
           20            45689.0   7626.0          14698.0  4220.0   
           21            43276.0   6533.0          13749.0  3525.0   
           22            41269.0   7731.0          13061.0  3236.0   
           23            29948.0   5427.0        